In [1]:
!pip install kagglehub
!pip install datasets
!pip install transformers
!pip install torch torchaudio
!pip install librosa
!pip install soundfile
!pip install scikit-learn
!pip install accelerate
!pip install evaluate
!pip install torchcodec
!apt-get install -y ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 25.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.


In [2]:
import os
import numpy as np
import pandas as pd
import torch
import evaluate
import kagglehub
from datasets import Dataset, Audio
from transformers import (
    Wav2Vec2FeatureExtractor,
    Wav2Vec2ForSequenceClassification,
    TrainingArguments,
    Trainer
)

In [ ]:
path = kagglehub.dataset_download("mozillaorg/common-voice")
print(f"Dataset path: {path}")

Using Colab cache for faster access to the 'common-voice' dataset.
Dataset path: /kaggle/input/common-voice


In [4]:
print("=" * 50)
print("📁 DATASET CONTENTS")
print("=" * 50)

for item in sorted(os.listdir(path)):
    item_path = os.path.join(path, item)
    if os.path.isdir(item_path):
        num_files = len(os.listdir(item_path))
        print(f"📁 {item}/ ({num_files} files)")
    else:
        size_mb = os.path.getsize(item_path) / (1024 * 1024)
        print(f"📄 {item} ({size_mb:.2f} MB)")

print("=" * 50)

📁 DATASET CONTENTS
📄 LICENSE.txt (0.00 MB)
📄 README.txt (0.00 MB)
📁 cv-invalid/ (1 files)
📄 cv-invalid.csv (2.25 MB)
📁 cv-other-dev/ (1 files)
📄 cv-other-dev.csv (0.26 MB)
📁 cv-other-test/ (1 files)
📄 cv-other-test.csv (0.26 MB)
📁 cv-other-train/ (1 files)
📄 cv-other-train.csv (12.80 MB)
📁 cv-valid-dev/ (1 files)
📄 cv-valid-dev.csv (0.37 MB)
📁 cv-valid-test/ (1 files)
📄 cv-valid-test.csv (0.37 MB)
📁 cv-valid-train/ (1 files)
📄 cv-valid-train.csv (18.20 MB)


In [ ]:
DATASET_PATH = path
SAMPLES_PER_ACCENT = 2000  # Accent Sampling
TARGET_ACCENTS = ["us", "australia", "england"]

# Label mappings
label2id = {"us": 0, "australia": 1, "england": 2}
id2label = {0: "us", 1: "australia", 2: "england"}

print(f" Configuration set")
print(f"   Target accents: {TARGET_ACCENTS}")
print(f"   Samples per accent: {SAMPLES_PER_ACCENT}")

 Configuration set
   Target accents: ['us', 'australia', 'england']
   Samples per accent: 2000


In [6]:
def load_and_filter_csv(csv_name, audio_folder):
    """Load CSV, filter for target accents, and construct audio paths"""
    csv_path = os.path.join(DATASET_PATH, csv_name)
    audio_dir = os.path.join(DATASET_PATH, audio_folder)

    # Load only needed columns
    df = pd.read_csv(csv_path, usecols=['filename', 'accent'])

    # Filter for target accents
    df = df[df['accent'].isin(TARGET_ACCENTS)]

    # Construct full audio paths
    df['audio'] = df['filename'].apply(lambda x: os.path.join(audio_dir, x))

    # Keep only files that exist
    df = df[df['audio'].apply(os.path.exists)]

    return df

print("Data loading function defined")

Data loading function defined


In [18]:
# Load training data from both valid and other sources
print("Loading training data...")
train_valid = load_and_filter_csv("cv-valid-train.csv", "cv-valid-train")
train_other = load_and_filter_csv("cv-other-train.csv", "cv-other-train")

# Combine training sources
train_data = pd.concat([train_valid, train_other], ignore_index=True)
print(f"Combined training data: {len(train_data)} samples")

# Balanced sampling - equal samples per accent
balanced_df = train_data.groupby('accent', group_keys=False).apply(
    lambda x: x.sample(n=min(len(x), SAMPLES_PER_ACCENT), random_state=42)
).reset_index(drop=True)

# Add numeric labels
balanced_df['label'] = balanced_df['accent'].map(label2id)

print(f"\n Balanced Dataset Created: {len(balanced_df)} total samples")
print(balanced_df['accent'].value_counts())

Loading training data...
Combined training data: 90942 samples

 Balanced Dataset Created: 6000 total samples
accent
australia    2000
england      2000
us           2000
Name: count, dtype: int64


/tmp/ipython-input-4585621.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  balanced_df = train_data.groupby('accent', group_keys=False).apply(


In [8]:
# Convert to Hugging Face Dataset
train_dataset = Dataset.from_pandas(balanced_df[['audio', 'label']], preserve_index=False)

# Cast audio column to resample to 16kHz
train_dataset = train_dataset.cast_column("audio", Audio(sampling_rate=16_000))

print(f"Training dataset created: {len(train_dataset)} samples")
print(train_dataset)

Training dataset created: 6000 samples
Dataset({
    features: ['audio', 'label'],
    num_rows: 6000
})


In [9]:
MODEL_NAME = "facebook/wav2vec2-base"

print(f"Loading Wav2Vec 2.0 model: {MODEL_NAME}")

# Load Wav2Vec2 Feature Extractor
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_NAME)

# Load Wav2Vec2 for Sequence Classification (3 accent classes)
model = Wav2Vec2ForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=3,
    label2id=label2id,
    id2label=id2label
)

# Freeze feature encoder for faster training
model.freeze_feature_encoder()

print(f"Wav2Vec 2.0 model loaded!")
print(f"   Total parameters: {model.num_parameters():,}")
print(f"   Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

Loading Wav2Vec 2.0 model: facebook/wav2vec2-base


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/159 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/configuration_utils.py:335: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/380M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec 2.0 model loaded!
   Total parameters: 94,569,347
   Trainable parameters: 90,368,899


In [ ]:
# Load accuracy metric
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {"accuracy": accuracy_metric.compute(predictions=predictions, references=labels)["accuracy"]}

print(" Metrics function defined")

 Metrics function defined


In [ ]:
def collate_fn(batch):
    batch = [x for x in batch if x["audio"] is not None and x["audio"]["array"] is not None]

    if not batch:
        return None

    # Process audio with feature extractor
    inputs = feature_extractor(
        [x["audio"]["array"] for x in batch],
        sampling_rate=16_000,
        max_length=80000,  # ~5 seconds at 16kHz
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )

    # Add labels
    inputs["labels"] = torch.tensor([x["label"] for x in batch])

    return inputs

print("Data collator defined")


Data collator defined


In [12]:
training_args = TrainingArguments(
    output_dir="./wav2vec2-accent-classifier",

    # Training hyperparameters
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,  # Effective batch size = 32
    learning_rate=5e-5,

    # Optimization for Colab
    fp16=True,                       # Mixed precision training
    gradient_checkpointing=True,     # Save memory

    # Logging
    logging_steps=10,
    logging_dir="./logs",

    # Other settings
    remove_unused_columns=False,     # Keep audio column
    save_strategy="epoch",
    load_best_model_at_end=False,
    report_to="none"
)

print("Training arguments configured")
print(f"   Epochs: {training_args.num_train_epochs}")
print(f"   Batch size: {training_args.per_device_train_batch_size}")
print(f"   Gradient accumulation: {training_args.gradient_accumulation_steps}")
print(f"   Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")


Training arguments configured
   Epochs: 3
   Batch size: 2
   Gradient accumulation: 16
   Effective batch size: 32


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    compute_metrics=compute_metrics
)

print("🚀 Starting training...")
print("=" * 50)

# Train the model
train_result = trainer.train()

print("=" * 50)
print("Training complete!")
print(f"   Training loss: {train_result.training_loss:.4f}")
print(f"   Training time: {train_result.metrics['train_runtime']:.1f} seconds")

🚀 Starting training...


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss
10,1.098600
20,1.033500
30,1.038000
40,0.973900
50,0.966700
60,0.873600
70,0.859800
80,1.031600
90,0.881400
100,0.923300


/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Training complete!
   Training loss: 0.5554
   Training time: 1055.9 seconds


In [ ]:
print("Loading test data...")
test_df = load_and_filter_csv("cv-valid-test.csv", "cv-valid-test")
test_df['label'] = test_df['accent'].map(label2id)

print(f"Test samples: {len(test_df)}")
print(test_df['accent'].value_counts())

# Create test dataset
test_dataset = Dataset.from_pandas(test_df[['audio', 'label']], preserve_index=False)
test_dataset = test_dataset.cast_column("audio", Audio(sampling_rate=16_000))

print(f"\n Test dataset created: {len(test_dataset)} samples")


Loading test data...
Test samples: 1025
accent
us           633
england      299
australia     93
Name: count, dtype: int64

 Test dataset created: 1025 samples


In [15]:
print("=" * 50)
print("EVALUATING ON TEST SET")
print("=" * 50)

results = trainer.evaluate(eval_dataset=test_dataset)

accuracy = results['eval_accuracy']

print(f"\n FINAL TEST RESULTS:")
print(f"   Accuracy: {accuracy:.2%}")
print(f"   Loss: {results['eval_loss']:.4f}")


EVALUATING ON TEST SET



 FINAL TEST RESULTS:
   Accuracy: 94.27%
   Loss: 0.3434


In [16]:
MODEL_SAVE_PATH = "./wav2vec2-accent-final"

trainer.save_model(MODEL_SAVE_PATH)
feature_extractor.save_pretrained(MODEL_SAVE_PATH)

print(f"Model saved to: {MODEL_SAVE_PATH}")

Model saved to: ./wav2vec2-accent-final


In [17]:
print("\n" + "=" * 50)
print(" AUDIO ACCENT CLASSIFICATION COMPLETE!")
print("=" * 50)



 AUDIO ACCENT CLASSIFICATION COMPLETE!
